In [ ]:
from cuda import cuda

def check_driver():
    # 1. Initialize the Driver
    err, = cuda.cuInit(0)
    if err != cuda.CUresult.CUDA_SUCCESS:
        print(f"❌ cuInit Failed: {err}")
        return

    # 2. Get Device Count
    err, count = cuda.cuDeviceGetCount()
    if err != cuda.CUresult.CUDA_SUCCESS:
        print(f"❌ GetCount Failed: {err}")
        return
    
    print(f"🟢 Driver Initialized. Found {count} Device(s).")

    # 3. Get Device Name
    for i in range(count):
        err, device = cuda.cuDeviceGet(i)
        err, name = cuda.cuDeviceGetName(128, device)
        # name is bytes, decode it
        print(f"   GPU {i}: {name.decode('utf-8').strip()}")
        
        # Check Compute Capability (Crucial for 5070/Blackwell)
        err, major = cuda.cuDeviceGetAttribute(
            cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR, device
        )
        err, minor = cuda.cuDeviceGetAttribute(
            cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR, device
        )
        print(f"          Compute Capability: {major}.{minor}")

check_driver()

##### Warpgroup

there exists a map $w: [0, T) -> [0, \lfloor {T/32} \rfloor)$ given by $t \mapsto \text {floor} (t/32)$

indeed $w(t)$ is the warp rank of the thread t. A warpgroup is 4 contigous warps, with the warp_rank of the first warp being a multiple of 4. 

This is exactly what it means, threads 0,1,...31 (w0), 32,33,,,63 (w1), 64 .... 95 (w2), 

96 ..... 128 (w3) 
is a wrap group. its the 0th warp group. 

The first question we ask is that if the tensor core unit is seperate, and we fire out async WGMMA operations,
why do we need so many threads? the answer is simple, the accumulator tile is too big to be held by the 
registers of one thread, and is instead split as fragments across the threads of a warpgroup, also the SM can paralelly issue exactly 4 warps of instructions at once, while it can concurrently issue upto 2048/32 wrarps. 









In [ ]:
"""
okay now lets look into the whole situation 

First the TMA, imagine we have a tensor map, descibing the shared memory and global memory layouts of our tensor
along with any swizzle or interleave we wanna do, abstract that as T_map 

The first thing we need to do, is to init a barrier on shared memory with the number of arriving threads, 
and then call the tma fency proxy 
"""

In [1]:
def warp_mma_atom(c_frag, a_frag, b_frag): 
  c_frag = c_frag + a_frag@b_frag
  

In [ ]:
c_frag = np.zeros()